In [1]:
import andi
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import losses, metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM

from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import load_model
import os
AD = andi.andi_datasets()



#LTSM network for learning  the Hurst exponet H, Normalized trajectories!!!

#Building the network
model_norm_long = Sequential()
#first layer: LSTM of dimension 64
model_norm_long.add(LSTM(250,
                return_sequences=True,
                dropout=0,
                input_shape=(None, 5)
                ))

model_norm_long.add(LSTM(50,
                dropout=0))

#Last layer, fully connected
model_norm_long.add(Dense(1))
model_norm_long.compile(optimizer='adam',
                loss='mse', 
                metrics=['mae'])

#Printing a summary of the built network
model_norm_long.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 250)         256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                60200     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 316,251
Trainable params: 316,251
Non-trainable params: 0
_________________________________________________________________


In [2]:
i = 425
N = 100000
X1, Y1, X2, Z, X3, Z = AD.andi_dataset(N = N, tasks = 1, dimensions = 1,
                                             min_T = i, max_T = i+1, load_dataset=True, path_datasets=str(i)+'master1')

X2, Y2, Z, Z, X3, Z = AD.andi_dataset(N = N, tasks = 1, dimensions = 1,
                                             min_T = i, max_T = i+1, load_dataset=True, path_datasets=str(i)+'master2')

X3, Y3, Z, Z, Z, Z = AD.andi_dataset(N = N, tasks = 1, dimensions = 1,
                                             min_T = i, max_T = i+1, load_dataset=True, path_datasets=str(i)+'master3')

X1[0] = np.diff(X1[0],axis=1)
X2[0] = np.diff(X2[0],axis=1)
X3[0] = np.diff(X3[0],axis=1)

data_tot=X1[0]
data_norm1 = np.array(( data_tot - np.mean(data_tot, axis=1).reshape(len(data_tot),1) ) / np.std(data_tot,axis=1).reshape(len(data_tot),1))
data_tot=X2[0]
data_norm2 = np.array(( data_tot - np.mean(data_tot, axis=1).reshape(len(data_tot),1) ) / np.std(data_tot,axis=1).reshape(len(data_tot),1))
data_tot=X3[0]
data_norm3 = np.array(( data_tot - np.mean(data_tot, axis=1).reshape(len(data_tot),1) ) / np.std(data_tot,axis=1).reshape(len(data_tot),1))


Y1 = np.array(Y1[0])
Y2 = np.array(Y2[0])
Y3 = np.array(Y3[0])

Creating a dataset for task(s) 1 and dimension(s) 1.
Creating a dataset for task(s) 1 and dimension(s) 1.
Creating a dataset for task(s) 1 and dimension(s) 1.


In [3]:
#Building the network
model_norm_long = Sequential()
#first layer: LSTM of dimension 64
model_norm_long.add(LSTM(250,
                return_sequences=True,
                recurrent_dropout=0.2,
                input_shape=(None, 4)
                ))

model_norm_long.add(LSTM(50,
                    dropout=0,
                    recurrent_dropout=0.2,
                    ))

#Last layer, fully connected
model_norm_long.add(Dense(1))
model_norm_long.compile(optimizer='adam',
                loss='mse', 
                metrics=['mae'])

#Printing a summary of the built network
model_norm_long.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, None, 250)         255000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                60200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 315,251
Trainable params: 315,251
Non-trainable params: 0
_________________________________________________________________


In [3]:
from tensorflow.keras.models import load_model
model_norm_long = load_model('checkpoint.h5')


In [ ]:
for n in range(10):

    
    for batch_size in [32, 128, 512, 2048,]:
        
        for repeat in range(5):
            traj_show = data_norm1
            label_show = Y1
            history_norm_long = model_norm_long.fit(traj_show.reshape(len(traj_show),int(i/4),4), 
                                    label_show, 
                                    epochs=1, 
                                    batch_size=batch_size,
                                    validation_split=0.1,
                                    shuffle=True,
                                    )
            traj_show = data_norm2
            label_show = Y2
            history_norm_long = model_norm_long.fit(traj_show.reshape(len(traj_show),int(i/4),4), 
                                    label_show, 
                                    epochs=1, 
                                    batch_size=batch_size,
                                    validation_split=0.1,
                                    shuffle=True,
                                    )
            traj_show = data_norm3
            label_show = Y3
            history_norm_long = model_norm_long.fit(traj_show.reshape(len(traj_show),int(i/4),4), 
                                    label_show, 
                                    epochs=1, 
                                    batch_size=batch_size,
                                    validation_split=0.1,
                                    shuffle=True,
                                    )
            model_norm_long.save('checkpoint.h5')
    model_norm_long.save('Clone_425_' + str(4) +'.h5')
        
        

Train on 90000 samples, validate on 10000 samples
53536/90000 [================>.............] - ETA: 4:15 - loss: 0.1369 - mae: 0.2840

In [5]:
import scipy.io as sci
sci.savemat('125trial.mat',{'traj': traj_show, 'label': label_show})